In [37]:
from gurobipy import Model, GRB

In [38]:
# Create a new model
m = Model("road_salt_storage")

In [39]:
# Parameters
n_locations = 15
n_wards = 44
max_capacity = 4500
min_salt_per_ward = 500
fixed_cost = 500000

In [40]:
# Function to calculate transportation cost
def calc_cni(n, i):
    return (n / 2) * (11 - (i / 4))

In [41]:
# Decision variables
x = m.addVars(n_locations, vtype=GRB.BINARY, name="x")
y = m.addVars(n_locations, n_wards, vtype=GRB.CONTINUOUS, name="y", lb=0)

In [42]:
# Objective function
m.setObjective(sum(fixed_cost * x[n] for n in range(n_locations)) +
               sum(calc_cni(n+1, i+1) * y[n,i] for n in range(n_locations) for i in range(n_wards)),
               GRB.MINIMIZE)

In [43]:
# Constraints

# Constraint a: Capacity limit at each site
m.addConstrs((sum(y[n, i] for i in range(n_wards)) <= max_capacity * x[n] for n in range(n_locations)), "capacity")

# Constraint b: Demand at each ward
m.addConstrs((sum(y[n, i] for n in range(n_locations)) >= min_salt_per_ward for i in range(n_wards)), "demand")

# Constraint c: At least 8 storage sites must be opened
m.addConstr(sum(x[n] for n in range(n_locations)) >= 8, "min_sites")

# Constraint d: Limitation on specific sites
specific_sites = [1, 3, 5, 7, 9, 12]  # Corrected to match problem statement
m.addConstr(sum(x[n] for n in specific_sites) <= 3, "specific_limit")

# Constraint e: Linked opening of sites 7 and 13
m.addConstr(x[6] <= x[12], "link_7_13")

# Constraint f: Exact two sites among 1, 5, 14
m.addConstr(x[0] + x[4] + x[13] == 2, "exact_two")

# Constraint g: Exclusive sites 2 and 12
m.addConstr(x[1] + x[11] <= 1, "exclusive_2_12")

# Constraint h: Conditional openings
m.addConstr(x[2] <= x[1], "cond_3_2")
m.addConstr(x[2] <= x[8], "cond_3_9")
m.addConstr(x[2] <= x[10], "cond_3_11")

<gurobi.Constr *Awaiting Model Update*>

In [44]:
# Optimize model
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 67 rows, 675 columns and 1369 nonzeros
Model fingerprint: 0xac3bcd01
Variable types: 660 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e-01, 5e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective 5650187.5000
Presolve time: 0.00s
Presolved: 67 rows, 675 columns, 1366 nonzeros
Found heuristic solution: objective 4565250.0000
Variable types: 660 continuous, 15 integer (15 binary)

Root relaxation: objective 4.126875e+06, 122 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 4126875.00    0    2 4565250.00 4126

In [45]:
# Print solution
if m.status == GRB.OPTIMAL:
    print('Optimal solution found:\n')
    for v in m.getVars():
        if v.x > 0:
            print(f"{v.varName} = {v.x}")

else:
    print('No optimal solution found.')

Optimal solution found:

x[0] = 1.0
x[1] = 1.0
x[2] = 1.0
x[3] = 1.0
x[4] = 1.0
x[5] = 1.0
x[8] = 1.0
x[10] = 1.0
y[0,0] = 500.0
y[0,1] = 500.0
y[0,2] = 500.0
y[0,3] = 500.0
y[0,4] = 500.0
y[0,5] = 500.0
y[0,6] = 500.0
y[0,7] = 500.0
y[0,8] = 500.0
y[1,9] = 500.0
y[1,10] = 500.0
y[1,11] = 500.0
y[1,12] = 500.0
y[1,13] = 500.0
y[1,14] = 500.0
y[1,15] = 500.0
y[1,16] = 500.0
y[1,17] = 500.0
y[2,18] = 500.0
y[2,19] = 500.0
y[2,20] = 500.0
y[2,21] = 500.0
y[2,22] = 500.0
y[2,23] = 500.0
y[2,24] = 500.0
y[2,25] = 500.0
y[2,26] = 500.0
y[3,27] = 500.0
y[3,28] = 500.0
y[3,29] = 500.0
y[3,30] = 500.0
y[3,31] = 500.0
y[3,32] = 500.0
y[3,33] = 500.0
y[3,34] = 500.0
y[3,35] = 500.0
y[4,36] = 500.0
y[4,37] = 500.0
y[4,38] = 500.0
y[4,39] = 500.0
y[4,40] = 500.0
y[4,41] = 500.0
y[4,42] = 500.0
y[4,43] = 1000.0
y[5,43] = 4500.0
y[8,43] = 4500.0
y[10,43] = 4500.0
